In [ ]:
from osgeo import gdal, osr
import pandas as pd
import numpy as np
from sunmapcreator_2015a import sunmapcreator_2015a
from Solweig_2015a_metdata_noload import Solweig_2015a_metdata_noload
from sebeworker import Worker
import geopandas as gpd
from pyproj import Transformer
from pvlib import solarposition
import pvlib
import math
import time
from PIL import Image
import statistics
start_time = time.time()

In [ ]:
# Input parameters

gdal_dsm = gdal.Open('dsm.tif')
slope = gdal.Open('dsm_reclassify_slope.tif')
aspect = gdal.Open('dsm_aspect.tif')
gdal_wh = gdal.Open('wheight.tif')
gdal_wa = gdal.Open('waspect.tif')
shapefile = gpd.read_file("panels_grid.shp")
for i in shapefile.index:
    shapefile.loc[i, 'ymin'] = int(shapefile.loc[i, 'geometry'].bounds[1])
pv_power = gpd.read_file("panels_grid.shp")
segments = gpd.read_file("segments.shp")

tz = 'Europe/Berlin'
lat, lon = 52.52, 13.4
UTC = 1
times = pd.date_range('2019-01-01 00:00:00', '2020-01-01', closed='left',freq='H', tz=tz)
solpos = pvlib.solarposition.spa_python(times, lat, lon)
solpos['DOY'] = solpos.index.dayofyear
solpos['hour'] = solpos.index.hour
d = 1.5
panel_length = 1.6
panel_width = 1
flat_slope = 40

pv_eff = 0.177
alpha = -0.0039
albedo = 0.2
psi = 0.03
calc_month = False

In [ ]:
# Input meteorological data should be specifically formatted. 
# This specific format can be created using UMEP -> Pre-processing -> Meteorological data -> Prepare existing data. 
metdata1 = np.loadtxt("weather_data.txt", skiprows=0, dtype=float).reshape((-1, 24))

In [ ]:
dsm = gdal_dsm.ReadAsArray().astype(np.float)
building_slope = slope.ReadAsArray().astype(np.float)
building_aspect = aspect.ReadAsArray().astype(np.float)
wheight = gdal_wh.ReadAsArray().astype(np.float)
waspect = gdal_wa.ReadAsArray().astype(np.float)

In [ ]:
sizex = dsm.shape[0]
sizey = dsm.shape[1]
width = gdal_dsm.RasterXSize
height = gdal_dsm.RasterYSize
geotransform = gdal_dsm.GetGeoTransform()
minx = geotransform[0]
miny = geotransform[3] + width * geotransform[4] + height * geotransform[5]
scale = 1 / geotransform[1]
voxelheight = geotransform[1]

In [ ]:
for i in range(metdata1.shape[0]):
    
    if metdata1[[i], 14] > 0:
        
        onlyglobal = 0
        usevegdem = 0
        calc_month = False
        vegdsm = 0
        vegdsm2 = 0
        output = {'energymonth': 0, 'energyyear': 1, 'suitmap': 0} 
        
        metdata = metdata1[[i], :]
        old_cs = osr.SpatialReference()
        dsm_ref = gdal_dsm.GetProjection()
        old_cs.ImportFromWkt(dsm_ref)
        
        new_cs = osr.SpatialReference()
        wgs84_wkt = """
        GEOGCS["WGS 84",
            DATUM["WGS_1984",
                SPHEROID["WGS 84",6378137,298.257223563,
                    AUTHORITY["EPSG","7030"]],
                AUTHORITY["EPSG","6326"]],
            PRIMEM["Greenwich",0,
                AUTHORITY["EPSG","8901"]],
            UNIT["degree",0.01745329251994328,
                AUTHORITY["EPSG","9122"]],
            AUTHORITY["EPSG","4326"]]"""
        new_cs.ImportFromWkt(wgs84_wkt)
        transform = osr.CoordinateTransformation(old_cs, new_cs)
        lonlat = transform.TransformPoint(minx, miny)
        gdalver = float(gdal.__version__[0])
        
        if gdalver == 3.:
            lon = lonlat[1] #changed to gdal 3
            lat = lonlat[0] #changed to gdal 3
        else:
            lon = lonlat[0] #changed to gdal 2
            lat = lonlat[1] #changed to gdal 2
        alt = np.median(dsm)
        
        if alt < 0:
            alt = 3
        location = {'longitude': lon, 'latitude': lat, 'altitude': alt}
        
        
        YYYY, altitude, azimuth, zen, jday, leafon, dectime, altmax, timestamp = Solweig_2015a_metdata_noload(metdata,location, UTC)
        
#         print(timestamp)
        
        radmatI, radmatD, radmatR = sunmapcreator_2015a(metdata, altitude, azimuth, 
                                                        onlyglobal, output, jday, albedo, location, zen)
        
        total, direct, diffuse = Worker(dsm, scale, building_slope,building_aspect, voxelheight, sizey, sizex, 
                        vegdsm, vegdsm2, wheight,waspect, albedo, psi, radmatI, radmatD, radmatR, usevegdem, calc_month)
        
        
        for i in shapefile.index:
            
            poly2 = shapefile.loc[i, 'geometry']
            bounds = poly2.bounds
            geo_transform = gdal_dsm.GetGeoTransform ()
            
            pixel_y = int((((bounds[0] + bounds[2]) / 2) - geo_transform[0]) // geo_transform[1])
            pixel_x = int((((bounds[1] + bounds[3]) / 2) - geo_transform[3]) // (geo_transform[5]))

            
            Glob_hor = total[pixel_x, pixel_y]
            Diff_hor = diffuse[pixel_x, pixel_y]
            Dir_hor = direct[pixel_x, pixel_y]
                
            
            if shapefile.loc[i, 'Slope'] == flat_slope:
                
                flat_segments = shapefile[(shapefile['segment_id'] == shapefile.loc[i, 'segment_id'])]
                n_r = pd.unique(flat_segments['ymin'])
                
                slope1 = shapefile.loc[i, 'Slope']
                aspect1 = shapefile.loc[i, 'Seg_Aspect']

                l = panel_length
                w =  panel_width
                
                
                day = solpos.loc[(solpos['DOY'] == int(metdata[:, 1]))]
                phi = day.loc[day.index.hour == int(metdata[:, 2]), 'azimuth'].iloc[0]
                gamma = day.loc[day.index.hour == int(metdata[:, 2]), 'elevation'].iloc[0]
                zenith = day.loc[day.index.hour == int(metdata[:, 2]), 'zenith'].iloc[0]
                aoi = pvlib.irradiance.aoi(slope1, aspect1, zenith, phi)
                
                if gamma > 0 :
                    lamda = math.cos(math.radians(phi - aspect1)) + (math.tan(math.radians(gamma)) / math.tan(math.radians(slope1)))
                    if lamda <= 0 :
                        f_s1 = 1
                        f_s2 = 1
                    else:
                        f_s1 = 0
                        x_l = (math.cos(math.radians(slope1)) + d/l) * math.sin(math.radians(phi - aspect1)) / lamda
                        y_l = 1 - ((math.cos(math.radians(slope1)) + d/l) * math.tan(math.radians(gamma)) / (lamda * math.sin(math.radians(slope1))))
                        if y_l <=0:
                            f_s2 = 0
                        elif abs(x_l) > (w / l):
                            f_s2 = 0
                        else:
                            f_s2 = (1 - (abs(x_l)/ (w / l))) * y_l
                    f_sky1 = (1 + math.cos(math.radians(slope1))) / 2
                    f_sky2 = (1 + math.cos(math.radians(slope1)) + (d/l) - math.sqrt(((math.sin(math.radians(slope1)))**2) + (d/l)**2)) / 2
                    
                    D1 = f_sky1 * Diff_hor
                    D2 = f_sky2 * Diff_hor
                    
                    A1 = albedo * Glob_hor * (1 - f_sky1)
                    A2 = albedo * Glob_hor * (1 - f_sky2)
                    
                    B = Dir_hor * (math.cos(math.radians(aoi)) / math.sin(math.radians(gamma)))
                    
                    G_s1 =  (1 - f_s1) * B + D1 + A1
                    G_s2 =  (1 - f_s2) * B + D2 + A2
                    
                    if shapefile.loc[i, 'ymin'] == n_r.min():
                        shapefile.loc[i, f"{timestamp}"] = G_s1
                        pv_power.loc[i, f"{timestamp}_DC"] = G_s1 * pv_eff * (1 + alpha * ((metdata[:, 11] + 0.05 * G_s1) - 25)) * 1.6
                    else:
                        shapefile.loc[i, f"{timestamp}"] = G_s2
                        pv_power.loc[i, f"{timestamp}_DC"] = G_s2 * pv_eff * (1 + alpha * ((metdata[:, 11] + 0.05 * G_s2) - 25)) * 1.6
            else:
                
                shapefile.loc[i, f"{timestamp}"] = Glob_hor
                pv_power.loc[i, f"{timestamp}_DC"] = Glob_hor * pv_eff * (1 + alpha * ((metdata[:, 11] + 0.05 * Glob_hor) - 25)) * 1.6
 

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time)) 
for i in range(shapefile.shape[0]):
    shapefile.loc[i, 'sum'] = shapefile.iloc[i,8:25].sum(axis=0)
    pv_power.loc[i, 'sum'] = pv_power.iloc[i,8:25].sum(axis=0)

outfp = r"IRR_panel.shp"
segments.to_file(outfp)
outfp = r"PVP_panel.shp"
pv_power.to_file(outfp)